In [8]:
import os
import pandas as pd
from zipfile import ZipFile
import re
from bs4 import BeautifulSoup

# Functions to Compile Scraped MixesDB data

In [9]:
def check_page(soup):
    ### Function to determine whether or not a page is scrapable
    if len(get_metadata(soup)[1]) == 0:
        return(False)
    else:
        return(True)
    

In [10]:
def get_file_details(soup):
    ### Gets set length
    details = soup.select('#mw-content-text > table')
    if len(details) > 0:
        details = details[0]
    else:
        details = soup.select('#RepeatFakeIncluded > table > tbody')
        if len(details) > 0:
            details = details[0]
        else:
            return(None)
    x =  re.search(pattern='([0-9]:[0-9][0-9]:[0-9][0-9])', string=details.getText())
    if x is None:
        x = re.search(pattern='This show or mix is ~[0-9] hour', string=details.getText())
        if x is None:
            return(None)
        else:
            x = re.search(pattern='[0-9]', string = x.group())
            return('%s:00:00' % x.group())
    return(x.group())
    

In [11]:
def get_metadata(soup):
    ### Retrieves metadata such as set title, DJ/artist, genre, etc
    title = soup.title.text
    metadata = []
    for i in soup.select("#mw-normal-catlinks > ul > li > a"):
        metadata.append(i.text)
    
    return(title, metadata)

In [12]:
def get_tracklist(soup, metadata):
    ### Retrieves tracklist using 3 different seach mechanisms to deal with differing formats
    tracklist = []
    if(('Tracklist: none' not in metadata)&('Fake' not in metadata)):
        ## Lemma1
        for i in soup.select('#mw-content-text > ol > li'):
            track = i.text.strip()
            tracklist.append(track)
        ## Lemma2
        if(len(tracklist) == 0):
            for i in soup.select('#mw-content-text > div.list > ol > li'):
                track = i.text.strip()
                tracklist.append(track)
        ## Lemma3
        if(len(tracklist) == 0):
            for i in soup.select('#mw-content-text > div.list > div'):
                track = i.text.strip()
                tracklist.append(track)

    return(tracklist)

In [13]:
def parse_mixesdb(file):
    ### Given an HTML file, checks to see if a setlist page and then extracts the relevant information
    soup = BeautifulSoup(open(file), 'lxml')
    if check_page(soup):
        metadata = get_metadata(soup)
        tracklist = get_tracklist(soup,metadata)
        if len(tracklist) > 0:
            file_details = get_file_details(soup)
        else:
            file_details = None
    else:
        return(None)
    if len(tracklist) == 0:
        return(None)
    else:
        return(file_details, metadata, tracklist)
        
        

In [14]:
# Parses Data to page 6000 something. Runs into an error, which is resolved in next cell
#parsed_data = []
#i = 0
#for file in os.listdir('MixesDB/')[i:]:
#    parsed_data.append(parse_mixesdb("MixesDB/%s" % file))
#    i += 1
#    if i % 1000 == 0:
#        print('Parsed %s pages' % i)
    

In [15]:
### Skip error and keep compiling
#i += 1
#for file in os.listdir('MixesDB/')[i:]:
#    parsed_data.append(parse_mixesdb("MixesDB/%s" % file))
#    i += 1
#    if i % 1000 == 0:
#        print('Parsed %s pages' % i)

In [16]:
# Take parsed data and clean it for Nonetype entries
#cleaned_parsed = []
#for i in range(0,len(parsed_data)):
#    if parsed_data[i] is None:
#        pass
#    else:
#        cleaned_parsed.append(parsed_data[i])
    


In [17]:
# Save data from above cells
#import pickle
#fileHandle = open('data/mixesdb_scraped.pckl', "wb")
#pickle.dump(cleaned_parsed, fileHandle)

## Load Data processed from above cells

In [19]:
import pickle
pkl_file = open('data/mixesdb_scraped.pckl', 'rb')
cleaned_parsed = pickle.load(pkl_file)

In [88]:
### Split our (semi) cleaned and parsed mixesdb data into lists for complete, incomplete, and other tracklists
complete = []
incomplete = []
exceptions = []
for i in range(0, len(cleaned_parsed)):
    if 'Tracklist: incomplete' in  cleaned_parsed[i][1][1]:
        incomplete.append(cleaned_parsed[i])
    elif 'Tracklist: complete' in  cleaned_parsed[i][1][1]:
        complete.append(cleaned_parsed[i])
    else:
        exceptions.append(cleaned_parsed[i])

# Processing MixesDB data into a data table

In [89]:
def get_track_timing(track):
    # Retrieves track timing within a set, if available.
    track_timing = re.search(pattern='^\[.*\] ', string=track)
    if track_timing is not None:
        return(track_timing.group().strip())
    

In [90]:
def get_track_label(track):
    # Retrieves label for a track, if available
    label_info = re.search(pattern=' \[.*\]$', string=track)
    if label_info is not None:
        return(label_info.group().strip())
    

In [91]:
def trim_track(track):
    # Deals with some of the track naming inconsistencies on MixesDB
    
    # First get rid of extra track order
    x = re.sub(pattern='^\[.*\] ', string=track, repl = '')
    # Then get rid of label info
    x = re.sub(pattern=' \[.*\]$', string=x, repl = '')
    return(x.strip())

In [92]:
def get_track_artist_and_name(track):
    # separates artist from track name, prints out tracks that don't follow the normal format
    
    split_track = track.split(' - ')
    if(len(split_track) == 1):
        artist = split_track[0]
        track_ = split_track[0]
    elif(len(split_track) == 2):
        artist = split_track[0]
        track_ = split_track[1]
    else:
        artist = split_track[0]
        track_ = split_track[1]
        print(track)
    return(artist, track_)

In [93]:
def get_set_duration(set):
    #Retrieves length of a set
    return(set[0])

In [94]:
def get_set_title(set):
    return(set[1][0].split('|')[0].strip())

In [95]:
def parse_set_metadata(set):
    return(set[1][1])

In [96]:
## Create sets to identify intersections

genres = ['House', 'Tech House', 'Techno', 'Dub Techno', 'Acid Techno', 'Acid House', 'Hard Trance', 'Dubstep', 'Trance',
         'Progressive', 'Electro Pop', 'Disco', 'World Music', 'Hardcore', 'Hardcore Techno', 'Progressive House',
         'Chill Out', 'Future Jazz', 'Drum & Bass', 'Jungle', 'Hip Hop', 'Various', 'Synth Pop', 'House (Old School)', 
         'Breakbeat', 'Pop', 'IDM', 'Garage', 'UK Garage', 'Electro', 'Funk', 'Soul', 'Deep House', 'Deep Techno', 
         'Minimal', 'Minimal Techno', 'Electronica', 'Reggae', 'Jazz', 'Ambient', 'Classical', 'Psytrance', 
          'Progressive Trance', 'Hardstyle', 'Freestyle', 'Grime', 'Experimental', 'Industrial', 'Breaks', 'Detroit Techno',
         'Vocal House', 'EBM', 'Hard Techno', 'Euro House', 'Downtempo', 'Dancehall', 'Rave', 'Leftfield', 'Dub', 'Acid',
         'Dance', 'Hard House']
genres = set(genres)
years = range(1980,2018)
year_list = []
for year in years:
    year_list.append(str(year))
years = set(year_list) 

In [104]:
def get_set_year(set_metadata, years):
    yearout = list(years.intersection(set_metadata))
    if len(yearout) == 1:
        return(yearout[0])
    elif len(yearout) > 1:
        print('WAT WAT WAT')
        return(yearout[0])
    else:
        return(None)

In [111]:
def extract_set_genres(set_metadata, genres):
    #Returns a list of genres from set metadata
    return(list(genres.intersection(set_metadata)))

In [156]:
def search_set_genres(genre_extract, genre_search):
    search_results = {}
    for i in genre_search:
        if re.search(pattern=i, string=str(genre_extract)):
            search_results['bool_S_%s' % i.replace(" ", "_")] = True
        else:
            search_results['bool_S_%s' % i.replace(" ", "_")] = False
    return(search_results)

In [194]:
def get_ordered_tracklist(set):
    genre_search = ['Techno', 'House', 'Tech House','Trance', 'Disco', 'Psytrance', 'Electro', 'IDM', 
                    'Drum & Bass', 'Hip Hop', 'Garage', 'Detroit', 'Old School',
                    'Hard', 'Progressive', 'Acid', 'Deep', 'Minimal', 'Dub', 'Industrial',
                   'Leftfield']
    tracklist = set[2]
    set_duration = get_set_duration(set)
    set_title = get_set_title(set)
    metadata = parse_set_metadata(set)
    tracklist_frames = []
    year = get_set_year(metadata, years)
    genre_extract = extract_set_genres(metadata, genres)
    genre_searched = search_set_genres(genre_extract, genre_search)
    i = 0
    for track in tracklist:
        trimmed = trim_track(track)
        artist_, track_ = get_track_artist_and_name(trimmed)
        x = {'track_order': i, 'track': track_, 'artist': artist_,  
                             'label': get_track_label(track), 'track_timing': get_track_timing(track),
                                'set_duration': set_duration, 'set_title':set_title, 'set_metadata':metadata,
                                'set_year':year, 'set_genres':genre_extract}
        y = {**x, **genre_searched}
        tracklist_frames.append(y)
        i += 1
    return(pd.DataFrame(tracklist_frames))

In [195]:
# Compile dataframe for complete tracklists
tracklist_dfs = []
for i in range(0, len(complete)):
    tracklist_dfs.append(get_ordered_tracklist(complete[i]))
complete_df = pd.concat(tracklist_dfs)
complete_df = complete_df.reset_index(drop = True)
print(len(complete_df))

Luna Set - You Got In My Way [Jupiter - 6.25467 AP}
Abel Ramos - Atasco vs Pryda - Aftermath (Markus Schulz Mashup)
The Beatmonkeys - How You Like Me Now? - Rico Tubbs (Gangsters' Remix)
Delroy Edwards - Untitled - B4
Paul Ritch - Split The Line - Dubfire Mix
Pascal Feos - Frank Leicher - One Nation
Funk D' Void - Flealife - Christian Smith Remix
Slam - Collecting Data - Steve Lawler Mix
The Junkies - Bosco - UGLH Remix
Mathias Vogt - Dreamed The Impossible - S. Garcia Funk Remix
Rino Cerone - Rilis - Re-Edited
Dapyk & Padberg - Island - Floirian Meindl Mix
Pascal Feos - Guido Schneider - Unterwasser 20
Pascal Feos - Makina - M. Meinhardt Remix
Laurent Garnier - Flashback - Christian Smith & Wehbba Mix
Trinity - Don't Stop Till You Get Enough - Joe Gibbs
Marcia Griffiths - Stepping Out Of Babylon - Skynote
Micky Dread - Roots Man Revival - Unknown
Micheal Campbell - Rockers Dub - Unknown
Gussie And Van Cleef - Magnificent 7 - Unknown
Nina Simone - A Little Suger In My Bowl - Phillips
C

E.R.P. - Irma - Frantic FLowers
Larry Heard - Inside Your Mind (This House Is Home) - Alleviated
Kevin Over - Surge - Fortek
Schatrax - #5 - Shatrax
Roberto Rodriguez - The Days We Lost - Frisbee Tracks
E.S.O.M. - Air - Emphasis Recordings
DK7 - Slipstream - Output
Alexander Robotnick - Obsession (Andy Blake's Analogue Obsession) - Dissident
Quince - For My Mr. (Sterac Remix) - Delsin
Hienostunut Sonni - No Body (Levon Vincent Remix) - Alexi Delano Limited
Burnski - Nosebonker - Dessous
Jens Lodén - First One (André Lodemann Remix) - Fine Art
Hienostunut Sonni - Sharped Dressed Acid Man (Remix) - Alexi Delano Limited
X2 - Photon - M>O>S Recordings
Cabin Fever - Acid Party - RKDS
Roman IV - Altes Testament - Playhouse
Analog~1 - Reduced - MCMLXV
Arturo Garces - Got A Problem (Where Were You - J Lettow's More Vibes Remix)
International Pony - Gothic Girl - Mense Dub
Robert M - Cafe De Paris - Joia
Vitalic - You Are My Sun - Different
Cobblestone Jazz - Dump Truck - Wagon Repair
Yuksek - 

The Dose - I'm Your Door - Fumakilla 22
Guy Gerber - Late Bloomers - Corlp012
Robag Wruhme Als Rolf Oksen - Dopamin - Fat030
Âme - Balandine - Innervisions11
Samuel L. Session - Can You Relate - Kklap003-6
J.T.C. - Take 'em Off - Cremejak01
Paul Woolford Presents Bobby Peru - Lies - Vis146
Denis Karimani - Strange But Cool - 2000 and One Remix - Curle-M01
Argy - Unreliable Virgin - Cocoon
Sascha Dive - Down Edit - Argy Remix - Musik059
Faze - O - Riding high
Ez Rollers - Cops Dont Like Us - Unknown
Dom & Roland - Can't Punish Me - Unknown
Classified - Drone - Unknown
London Elektricity - Wishing Well - Hospital
Gwen McCrae - 90% Of Me Is You - Cat
Main Source - Just Hanging Out - Wild Pitch
Sister Nancy - Bam Bam - Fat City
Alpha Omega - Orian Skyes - Reinforce Records
Konk - Machine - Dog Brothers
Beat Less - Latin Aire - Ubiquity
Flying Fish - Lucy's Song - Cookin' Records
Sabu - Aurora Borealis - Columbia
Super All Stars - Ban-con-tim - Caimen Records
Ethiopian Quintet - Raina - Wor

Markus Schulz feat. Andy Moor - Daydream - Lemon & Einar K Remix - Coldharbour Recordings
Med vs Neil Bamford - One - Defcon Audio Rework - Abora Recordings
DJ Choose & Michael Splint - DJs Take Control - DJ Choose Remix - Nukleuz Records
Virunga - At Last - Original Mix - Mondo Records
Nitrous Oxide - Downforce (Mike Shiver Remix) vs Whirlpool - Under The Sun (Solarstone Remix) (B&B Mashup)
Loco Dice - Menina Brasiliera X Tyree Cooper - I Fear The Night (Soul Clap Tool)
D - T3ch - When It Was Good
Solarity - Terminal 6 (06:27)[00:00] 01. Yvel & Tristan - Heart & Soul (TV Maniacs Mix)
Westpark Unit - Jam Hot - Farside
Chez Damier - Why - Mojuba
Rainer Trüby - Ayers Rock - Compost
JC Freaks - Dub Praise (The Revenge Remix) - Phonica
San Soda - Het Zwarte Kanon - We Play House
Ribn - This Feeling - Mild Pitch
Westpark Unit - More Forever - Farside
Robert Bosco - Dub Piano - Be As One
Henry L - The Other Day At The Basement (Marcus Worgull's Flugskompensator Dub) - Farside
Visti & Meyland

Dimitri From Paris - Sacrebleu (Yellow - YP CD 011)
Digital Mama - Dig This (Overdose - DMD Dose 002 ltd)
Ken Ishii - Overlap (R & S - RS 96107)
Earth Nation - Educate It (Low Spirit - DMD LS 201 MS)
Ken Ishii - Overlap (R & S - RS 96107)
Talla 2XLC - Is Anybody Out There (Low Spirit - DMD LS 201 MS)
Mysteryman - X-Or (Overdrive - OVER-X-PO 05)
Talla 2XLC - Is Anybody Out There (Low Spirit - DMD LS 201 MS)
Mysteryman - Vibes (Overdrive - OVER-X-PO 05)
Franky Jones - Nightflight (Bonzai - BRLP 96001)
Black & Brown - Lick It (D:Tour - DET 120010)
Creation (Nuclear EP) - Serious (One Thousand - ots 1002)
Pacific Rhythm LP - Track 1 (Harthouse - HH SP 010 CD)
Bubblegum Crisis - Syncrasy (Overdrive - OVER 093)
Pacific Rhythm LP - Track 2 (Harthouse - HH SP 010 CD)
Pili Pili 96 - Psycho Drum Mix (Intercord - INT 128.151)
Franky Jones - Funki Bizznizz (Bonzai - BRLP 96001)
02 - Funk D'Void - Bad Coffee
03 - Abstract Soul - Intent
04 - DJ Fallovie - Party Animal
05 - Function - Anosification
0

...

Run Dmc - My Adidis - Profile
Run Dmc - King Of Rock - 4th & Broadway
Run Dmc - Sucker Mc's - Profile
Run Dmc - Peter Piper - Profile
Run Dmc - What's It All About - Profile
Multicast - Foehn - Obliq
Rapaccioli - Klien - Vertical Form
Anthony Burgess Reads - Clockwork Orange - Caedmon
Darkglobe - Re-ascent - Unknown
Chapter One - Arse In Gear - Breakneck
K-paul - If U Know What I Mean - Loud &slow
Chemical Reaction Food - Are You Mad - Loud &slow
The Bell Café Band - Gorilla With A Gun - 2000 Dynamite Joint
Jeff Noon David Toop - Scratching Mad Magic - Sulphur
Jimpster - Weft & Warp - Kudos
Opiate - Toothpaste - April
Lalo Schifrin - Dirty Harry Theme - Aleph Records
Yves Hyatt - Path To Ascension - Unknown
Marschmellows - King O Trash - Compost
DJ Cam Kenny Dope Mix - Success - Inflamble
A Tribe Called Quest - Give Me - White Label
DJ Honda & Mos Def - Travlin Man - White Label
Damien - JR Gong' Marley - Welcome to Jamrock
Surgeon - First - Tresor
Katie Kissoon - You're The One (You'r

True Pseudo - Freakin - Me Out (ULTRNX Remix / NAPT Edit)
Lambchop - Up With The People - City Slang
St Germain - Montego Bay - Blue Note
G Corp - Dub 3000 - Different Drummer
Sofa Surfers - Sweat - Klien
Shawn Lee - The Sun - We Love You
Live Human - Lesson #7 - Matador
ESG - Bam Bam Jam - Unknown
ESG - Get Funky - Unknown
ESG - You And Me - Unknown
ESG - Crash - Unknown
ESG - Your No Good - Unknown
ESG - Earn It - Unknown
ESG - Scream And Shout - Unknown
ESG - Dance - Unknown
ESG - Come Away - Unknown
Art Farmer - Soul Sides - Mainstream
Skitz + Julie Dexter - Be - Unknown
Riddla - Deadline 2 - Unknown
Landslide + Victor Davies - Tumbling - Hospital
Lianne Carrol - The Trap - Hospital
Yukihiro Fukutomi - Mirrors - Hospital
St Germain - Rose Rouge - Blue Note
Marcus Intalex - Taking Over Me - Hospital
Opaque - Closing The Loop - Unknown
Barry Morgan - Samba Street - Unknown
Landslide - Fortuna - Hospital
Space Clique - Exit 1 : Luna Park - Unknown
London Elektricity - Incurable - Hosp

The Beatles - Jessie's Dream - Unknown
Raymond Scott - Bendix 2; The Tomorrow People - Basta
Joe Meek & The Blue Men - I Hear A New World - RPM Records
Automotivation 2 - Cabaret Voltaire - Sheffield
Birth - Raudive - Ealing
It Slipped Her Mind - Sandra Electronics - Downwards
Victims - Tropic Of Cancer - Downwards
Revox Love - Machinagraph - Sheffield
TV AD - Machinagraph - Sheffield
Do The Mussolini (Head Kick) - Cabaret Voltaire - Sheffield
The Set Up - Cabaret Voltaire - Sheffield
Entrance (Machinagraph Edit) - Raudive - Ealing
Chemistry (Machinagraph Edit) - Antonym - Downwards
Spread The Virus - Cabaret Voltaire - Sheffield
New Girls Neutron - Vice Versa - Sheffield
Being Boiled - Human League - Sheffield
Lindsey Buckingham - Big Love[Eagle Vision - EV303749]
Nalin & Kane - Open Your Eyes (Markus Schulz & Elevation Dub Mix) / Lost Tribe - Gamemaster (Acappella)
Narcotik - Blue / Age of Love - The Age of Love (Acappella)
Age of Love - The Age of Love (Watch Out For Stella Club Mix

TDr. - Squelch - Part II - Oxygen
Dogzilla - Without You - Maelstrom
DJ Hal - New Era - Rolex
Ambassador - The Fade - Fade Mix - Smash Rec
Jaia - Orchestra 2,0 - Minilogue Remix - Digital Structures
Paul van Dyk Feat. Rea Garvey - Let Go (Paul van Dyk Club) vs Paul van Dyk Feat. Second Sun - Crush (Acapella) (Paul van Dyk Mashup)
Stoneface & Terminal - Volcano (2am Mix) vs Lenny Kravitz - Believe In Me (Mind One Remix) (Paul van Dyk Mashup)
Angelo - Badalamenti - Brain's Birthday
Sleezy D - I've Lost Control - Space Mix - Trax
K. Alexi - Vertigo - Transmat
Jesse Saunders - On And On Rhythm Tracks - Jes Say Records
Jamie Principle - It's A Cold World - Trax
Jesse Saunders - On And On Tracks - Jes Say Records
The Garden Of Eden - The Serpent In The Garden - Pepper Records
Baby Ford - Ford Trax - Torso Records
Risque III - Essence Of A Dream - Stride Records
Master C & J - Dub Love - Trax
Mr. Fingers - Children At Play - Jack Trax
Baby Ford - Crashing - Ford Trax - Torso Records
Robert Ow

DB-X - Blip - Accelerate
69 - My Machines - Planet E
Rob Lenarduzzi - Slingshot - Jacktripper
Etat Solid - Think About It - !K7
The Field - The Deal - Kompakt
Jack Master - Bang The Box - Basement Trax
Delia & Gavin - Rise (DFA Remix) - DFA
Plastik Man - Spastik - Plus 8
Tyree Cooper - T.C.X. - Cosmic
Farley Jamckmaster Funk - Farley Knows House - Trax
Sheath - His Sheath - Proptronix
Navario Sauro - Lancia Delta Inerale 16V - Scandinavia
DJ Pierre - Land Of Confusion Remix - Secret Mixes
Tyree - Video Crash - Rockin' House
V-Room - V-Shaped - Plus 8
Millsart - Humana - Axis
Rex Sepulveda - Ikon Co Qui - D
Astrospider - Ritmista Remix - Wonka
Senor Coconut - Behind The Mask (Al Usher Remix) - Essay
Robert Hood - One Touch - Axis
33 1/3 Queen - Searchin' - Nu Groove
K. Alexi Shelby - All For Lee-Sah - Transmat
Metro Area - Rhythm Reel #6 - Environ
FUSE - Mantrax - Plus 8
FUSE - Nitedrive - Plus 8
Kenny Larkin - Life Goes On - R&S
Invisible Conga Band - Weird Pains - Italians Do It Bette

In [196]:
# Compile dataframe for incomplete tracklists
tracklist_dfs = []
for i in range(0, len(incomplete)):
    tracklist_dfs.append(get_ordered_tracklist(incomplete[i]))
incomplete_df = pd.concat(tracklist_dfs)
incomplete_df = incomplete_df.reset_index(drop = True)
print(len(incomplete_df))

Zap - Ringo & Nobby Uno - Swat
File:2009-07-09 - The Mole - Noice! Podcast 48 - ID 50.mp3?
Young MC - The Fastest Rhyme - My Name Is Young (Vocal)
Evolution - Walking On Fire / Celeda - Be Yourself (Danny Tenaglia's Reedit)
13. Wild Motherfuckers - Fuck It Up/Overload - The Anthem (Mash Up)
George Benson - Footin - It (A&M 1968)
Peter Gennaro - Annie OST - Hard Knock Life
IEmerge - ITF 2002 Advancements East Coast Champ - Take Em To War (Quick Interlude)
Slick Rick - Sittin - In My Car (Def Jam 1995)
GQ - Sittin - In My Car
The Pharcyde - Passin - Me By
Michael McDonald - I Keep Forgettin - (Warner 1986)
File:1983-06-11 - Ben Liebrand - In The Mix - ID.mp3?
Brian Eno - David Byrne - The Carrier
Aneesh Gera & Fernando - Eye For An Eye (Omar Bicane - Intro Edit)
The Overlords - God's Eye - God's Eye On Goa (Bionizer Remix)
Pascal FEOS - Makina vs. Junior Boys - Like A Child (Carl Craig Remix)
Mobb Deep - Shook Ones / Notorious B.I.G. - Unbelievable Blend
File:2007-08-11 - Derrick May @ C

Aphex - Unreleases (New Track - Singapore / Dublin)
Harry - Choo Choo' Romero - Say My Name (Switch Remix)
Harry - Choo Choo' Romero - Back In The Day (Acapella)
Erick Morillo - I'm Still Waiting (Harry - Choo Choo' Romero Remix)
Wahoo - Make - Em Shake It (Sandy Rivera Remix)
Dave McCullen - B! - Ch
Lil Mo - Ying Yang - Reach
Cosmic Gate Feat Denise Rivera - Body Of Conflict vs Nenes - Teksha (Filo & Peri Mashup)
Scritti Politti - Perfect Way (Version - Extended Mix)
Antidote - Transcentral with Daft Punk - Around The World
Run - D.M.C. - Rock Box
Mark August - Oxytocin Mezzotinto - MEZZO 01]
Culture Beat - Cherry Lips (Instrumental Magic) [Epic - 49 73170[
Klaus Goulart - Turbulence - Gufman Celebration
Kamasutra - Running Away Or Nicole (Running Away - E-Smooth Mix)
Marco V - Atlanta 02/02/02 (With Vocals From Kid Vicious - Proceed)
Malik Flavors - Untitled [Stones Throw - Unreleased]
 secret record set*
Madvillain Feat. Stacey Epps - Eye [demo mix) [Stones Throw - Unreleased]
 secr

File:2015-03-05 - Walker Barnard - Ibiza Voice Podcast - Last ID.ogg?
Atlantic Starr - Silver Shadow (Cool, Calm, Collected - U.S. Club Mix)
Rejected - Let's Go Juno (District One Remix) + Wink - Superfreak (Accappella)
Licking Lyrics - Just - Ice
Damon Wild & Tim Taylor - Bang The Acid - Rebanged! (Echoplex Remix)
File:2001 - Theo Parrish - These Days & Times Part 1-05.mp3?
Marshall Jefferson And On The House - Move Your Body - House Music Anthem
Marco V Feat. Khashassi - Solitary Confinement + Marco V - Simulated 2010
G-Man - Quo Vadis - i220 (1999)
Think Its Johnny Vicious Frozen Bass - The Swing (Which Samples Bohannon - Let's Start The Dance)
File:2009-11-13 - Marcel Woods - Power Partyzone - ID 03.mp3?
Pan - Pot - Charly
Plan Tec - Espias Psiquicios - (Jonas Kopp Psiquic Remix)
Stereocalypse - Lace Star [Siamese - SIAMESE002]﻿
Zakes Bantwini - Wasting My Time (Dan Ghenacia Remix - The Martinez Brothers Edit)
Some Horrible Trainwrecking - Kevin Gorman - Insomnia (Joseph Capriati R

JB - Luv Urself + Wwings - Signal
Jerome Isma-Ae - Hold That Sucker Down vs Vortechtral - Round The Block (Reaky Mashup)
Peppelino - Scitec vs The Ones - Flawless (Reaky Mashup)
Peppelino - Barcelona vs Industrialyzer - Kosmos (Reaky Mashup)
Aaron Arce - LA Transit - Sublevel
Sun Electric - Toninas - Ricardo Villalobos Remix
System Seven - Space Bird - Dubfire Remix
Radio Slave - Grindhouse - Danton Eeprom Vocal Version
Jerome Sydenham & Tiger Stripes - Elevation - Radio Slave One More Kiss Remix
Partial Arts - Telescope - Radioslave Remix
Saint Germain - Rose Rouge - Takk Su Remix (till end)
Paul van Dyk - For An Angel !Special Live Remix) + - Live Keyboardstrings]
Goldie - Chico - Death Of A Rock Star
Real McCoy - Automatic Lover (Armand's NYC - Miami Mix)
Harry - Choo Choo - Romero - What Happened? (Acapella)
Jorn van Deynhoven & Temple One - Halo (Jorn van Deynhoven Mix) / Matt Darey Pres. Urban Astronauts Feat. Kate Louise Smith - See The Sun (Acapella)
Alter Ego - Lycra [Harthous

Eight Miles High - Conversation - Klang 009
Jeff Mills - Java - Purpose Maker 001
Phuture - We are Phuture - Primate 001
Wishmountain - Radio - Evolution 031
Dullah Beatz - Dullah Circles (Preditah - Circles Remix)
The Overlords - Sundown (Sunshine And The 909 - Babymix)
Jimmy Van M - E.C.I - P.S
File:Chuggin Edits - The Puzzle Project Sessions Vol.1 - ID 43.ogg?
File:Chuggin Edits - The Puzzle Project Sessions Vol.1 - ID 62.ogg?
Romanthony pres. Naida - Do You Think You Can Love Me (Tru-Skool Vocal #14) - Glasgow Underground 15
Ujuzi - Sonny Daye  (Restless Soulful Mix) - Pan 7
4hero - Cosmic Tree  - Talkin' Loud
London Elektricity - Song In The Key Of Knife  - Hospital 7
Omni Trio - Meltdown  - Moving Shadow 121
Digital - X-Press  - Creative Source 20
Sylex - Washing  - Future Talk 4
Ed Rush & Optical - Crisis  - Virus 3
Future Homosapiens - 4th Dimension Blues  (20:20 Vision Remix) - Galactic Disco 13
Terry Callier - Love Theme From Spartacus (Roy's Steppers Delight) - Talkin' Loud 

...

The Killers - Spaceman (Sander Van Doorn Mix - Part 2)
Sander Van Doorn - Get Down & Sander Van Doorn vs Marco V - What Say
Tricky Disco - Tricky Disco (The Martin Brothers - Thizzy Disco' Remix)
Sergio Fernandez - Thank You - Insert Coin
Santos - Primitive - Cannible (Reboot)
Celeda - The Underground (ID Remix) vs Danny Tenaglia - The Chant
DJ Food - Ageless Young Rebel - Ninja Tune
+ Paul van Dyk Feat. Rea Garvey - Let Go (Vandit Club By Paul van Dyk / Paul van Dyk Club Mix) vs Paul van Dyk Feat. Second Sun - Crush (Acapella) (Paul van Dyk Mashup)
De - Lacy - Hideaway
Westpark Unit - Jam Hot - Farside
Chez Damier - Why - Mojuba
Rainer Trüby - Ayers Rock - Compost
JC Freaks - Dub Praise (The Revenge Remix) - Phonica
San Soda - Het Zwarte Kanon - We Play House
Ribn - This Feeling - Mild Pitch
Westpark Unit - More Forever - Farside
Robert Bosco - Dub Piano - Be As One
Henry L - The Other Day At The Basement (Marcus Worgull's Flugskompensator Dub) - Farside
Visti & Meyland - Yes Maam (Al

File:2015-02-28 - Dusky @ The Social - ID 00.ogg"Sex hier, ja"?
Andrew Richley & Ryan Rivera - Trackman (Chris Liebing Remix) [Fine Audio Recordings - AUDIO 09r)
Brain Unltd. - Body - Mind
Brett Johnson - Two Cents (Massi DL 100 Resampling Remix) vs Ruben Blades - G.D.B.D. (Acappella)
Paul Kalkbrenner - Valentin Van Corner - Black Room
Ramin - Brainticket [Dance Ecstasy 2001 - 2012[
Tribute Mashup to Mr. Magic (RIP)
 Whodini - Magics Wand MC Shan - The Bridge Juice Crew All Stars - Oh My Goodness Biz Markie - Nobody Beats The Biz Marley Mar - We Cut So Fresh MC Shan - Down By Law
Ultradyne - Agony - Be Still
Terence Fixmer - Electrostatic - Audio Remix
Infiniti - Game One - Steve Rachmad Remix
Samuel L Session - The Stick Up - Remix 2
Dash Berlin - When You Were Around (Feat. Kate Walsh - Club Mix)
Afrika Bambaataa And Westbam Proudly Pres. I.F.O. - Agharta - The City Of Shamballa
+ File:2004-09-23 - Expect The Unexpected, Part 3, ID 16.mp3Track over Phuture - Acid Trax
VS - Flamingo H

Old Dirty Bastard - Shimmy Shimmy Ya (Acapella) - Elektra
Mr. Scruff - Spandex Man - Ninja Tune
Jurassic 5 - Swing Set - Interscope
This Kid Named Mile - Ring Of Fire - Alatac
DJ Shadow - 6 Days (Soulwax Remix) - Island
Tuff Love - Haus Am Wald (Drake Scorpio & Mrbenn Mix) - Leisure Recordings
Mr. Benn - The Jump - Leisure Recordings
Mark Ronson - Toxic (Paul Nice Remix) - Sony
Jumbonics - Last Night - Tru Thoughts
A-Skillz & Nick Thauer - Rock The Spot - Tru Thoughts
Architecture In Helsinki - Heart It Races (A-Trak Remix) - Polyvinyl
Chemical Bros - Music Reponse - Virgin
Daft Punk - Da Funk - EMI
Justin Timberlake - Sexy Back (Acapella) - Jive
Full Phat - Fat Burdy - Refo
Chromeo - Tenderoni - Turbo
Diplo - Work Never Over - Mad Decent
Dizzee Rascal - Sirens (Paul Devro Edit) - XL
Bonde Do Role - Solta O Frango - Domino
LCD Soundsystem - Disco Infiltrator - DFA
Enur Feat. Natasha Calabria - Calabria (Hatchmatik Clubabria Remix) - White
Kid Sister - Control (Sinden's Calabria Remix) 

Eddie C - Tell Me (Tornado Wallace Mix) - Home Taping Is Killing Music
Rumble Feat. Duck Rules - Tricky Tricky vs Zombie Nation - Kernkraft 400 (Vocals)
Podcast Player - John O'Callaghan: Jochen Miller - Eclipse
Kiki & Silversurfer - Shake Off (Justin-Robertson's Revtone Dub) [Feat. Captain Comatose] - Crosstown Rebels
Jordan Rivera - Chambo vs JUANiTO - El Mapale (Roger Sancehz Mashup)
Andy Rojas - Moloko vs Leon Benesty & Brothers In The Booth - One Day (Roger Sanchez Mashup)
Jeremy Bass - Work vs Roger Sanchez - Turn On The Music (Acapella) (Roger Sanchez Mashup)
Phantom Regiment - BEA - (Counterpart Remix)
Hardhouse Banton - Reign (Roska's - Ard Food Mix)
Zinc - Zinc - Blunt Edge
Shortstuff - Shortstuff - Tripped Up
DJ ESP - Love + Hell - Let No Man Jack
Loleatta Holloway - Dreamin'(Acapella) + Colonel Abrams - Music Is The Answer (Beat)
File:1997 - Stacey Pullen - Radio Nova 03.ogg?
File:1997 - Stacey Pullen - Radio Nova 05.ogg?
File:1997 - Stacey Pullen - Radio Nova 08.ogg?
File:

Krash - Fighters vs DJ Lukas - Fucking Crazy (Mondello Mashup)
Trentemöller - Moan - Poker Flat
DBX - Blip - Accelerate
Schneider, Galluzzi & Schirmacher - Albertino - Cadenza
Brothers' Vibe - El Baile (Acapella) - SOM
Alex Picone - Furby - Cadenza
Mute & Metazoo - Sigh (2000 and One Remix) + Ralphi Rosario Feat. Rosabel - La Puta (Acapella)
Mute & Metazoo - Sigh - Cray Labworks
Rosabel - La Puta (Acapella) - Groovilicious
Inner City - Good Life (Carl Craig Acap. Remix) - KMS
Dennis Ferrer - Transitions + Ricardo Villalobos - Japanese Drums at the Afterhours + ID
Static Drum - Daddy rich (Original_Mix) + M83 - In Church
Butch - On The Line (Oxia Remix) + Yiorgos Mangas - Gia Tous Anthropous Pou Agapo
Jose Padilla - Souvenir - Bossa Rosa (Vocal)
Sly One - This Late Stage (Aly & Fila Mix) vs Bryan Kearney - You Will Never Be Forgotten (Activa Mix) vs The Doppler Effect (Vocals) (John Askew Mashup)
Argy - The Storm + Submission - Women Beat Their Men
Samuel L Session - A Bastards Work Is 

In [197]:
complete_df['bool_S_complete'] = True
incomplete_df['bool_S_complete'] = False
concat_these = [complete_df, incomplete_df]
mixesdb_df = pd.concat(concat_these)

In [199]:
mixesdb_df[mixesdb_df['bool_S_Disco'] == True]

,artist,bool_S_Acid,bool_S_Deep,bool_S_Detroit,bool_S_Disco,bool_S_Drum_&_Bass,bool_S_Dub,bool_S_Electro,bool_S_Garage,bool_S_Hard,...,label,set_duration,set_genres,set_metadata,set_title,set_year,track,track_order,track_timing,bool_S_complete
18,Isaac Hayes,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,I Can't Turn Around (Edit),0,None,True
19,Mike T,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Do It Any Way You Wanna,1,None,True
20,The Whispers,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,One For The Money (Edit),2,None,True
21,Loleatta Holloway,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Hit And Run,3,None,True
22,The Dells,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,No Way Back (Edit),4,None,True
23,J.M. Silk,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Music Is The Key (Basement Key),5,None,True
24,Beats + Colonel Abrams,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Trapped (Acapella),6,None,True
25,Chip E.,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,MB Dance,7,None,True
26,Ron Hardy,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Sensation (Unreleased Version),8,None,True
27,Direct Current,False,False,False,True,False,False,False,False,False,...,None,0:45:14,[Disco],"[1985, Ron Hardy, Muzic Box, DHP, Disco, Track...","1985 - Ron Hardy @ Muzic Box, Chicago (DHP 641)",1985,Everybody Here Must Party,9,None,True


In [200]:
mixesdb_df.to_csv('mixesdb_all_sets_featurized.csv')

In [182]:
# Get artist counts to find most popular track and artist
complete_counts_artist = mixesdb_df.artist.value_counts()
complete_counts_track = mixesdb_df.track.value_counts()

In [193]:
print("Most charted artists\n")
print(complete_counts_artist.head(n = 25))
print('\n Most charted tracks\n')
print(complete_counts_track.head(n = 25))

Most charted artists

?                            55666
Jeff Mills                     993
Skream                         726
Slam                           667
Robert Hood                    661
Chris Liebing                  605
Dave Clarke                    577
Sander van Doorn               560
Surgeon                        557
Ferry Corsten                  543
Joey Beltram                   519
Green Velvet                   507
Daft Punk                      500
Planetary Assault Systems      499
Radio Slave                    495
Underworld                     477
Ben Sims                       476
Calibre                        471
Depeche Mode                   468
Joris Voorn                    467
Laurent Garnier                463
Marco Bailey                   462
Marco V                        441
Gary Beck                      431
Adam Beyer                     429
Name: artist, dtype: int64

 Most charted tracks

?                   56495
Untitled             1567
I

?                            55666
Jeff Mills                     993
Skream                         726
Slam                           667
Robert Hood                    661
Chris Liebing                  605
Dave Clarke                    577
Sander van Doorn               560
Surgeon                        557
Ferry Corsten                  543
Joey Beltram                   519
Green Velvet                   507
Daft Punk                      500
Planetary Assault Systems      499
Radio Slave                    495
Underworld                     477
Ben Sims                       476
Calibre                        471
Depeche Mode                   468
Joris Voorn                    467
Laurent Garnier                463
Marco Bailey                   462
Marco V                        441
Gary Beck                      431
Adam Beyer                     429
Name: artist, dtype: int64

?                                                  55666
Jeff Mills                                           993
Skream                                               726
Slam                                                 667
Robert Hood                                          661
Chris Liebing                                        605
Dave Clarke                                          577
Sander van Doorn                                     560
Surgeon                                              557
Ferry Corsten                                        543
Joey Beltram                                         519
Green Velvet                                         507
Daft Punk                                            500
Planetary Assault Systems                            499
Radio Slave                                          495
Underworld                                           477
Ben Sims                                             476
Calibre                        

In [62]:
def parse_discogs_tracks(release):
    track_frames = []
    tracklist = release.tracklist
    release_title = release.title
    release_artist = release.artists[0].name
    genres = release.genres
    styles = release.styles
    release_date = release.data.get('released_formatted')
    descriptions = release.formats[0].get("descriptions")
    year = release.year
    country = release.country
    cat_no = release.data.get('catno')
    
    for track in tracklist:
        if len(track.artists) == 0:
            track_artist = release_artist
        else:
            track_artist = track.artists[0].name
        track_frames.append({'position': track.position, 'track': track.title, 'artist': track_artist, 
                             'release_artist': release_artist, 'length': track.duration,'release': release_title, 
                             'genre': genres, 'styles': styles, 'year': year, "release_date": release_date, 
                             'country': country,'cat_no': cat_no, 'release_descriptions': descriptions, 
                             'label': release.labels[0].name})
    return(pd.DataFrame(track_frames))